# Level 2: Simple Agentic 

This tutorial is designed to give new users an overview of how to use llama-stack's builtin agent framework effectively. It demonstrates basic single-tool usage and provides guidance on switching between a local development environment and a remote kubernetes cluster.

## Overview

This tutorial walks you through how to build your own AI agent who can search the web following these steps:
1. Connecting to a llama-stack server (remote or local).
2. Configuring an agent for tool use.
3. Running the agent.


## Prerequisites

Before starting, ensure you have the following:
- Access to a remote cluster or a local Podman setup.
- A Tavily API key is required. You can register for one at [https://tavily.com/](https://tavily.com/).


## Building a Web Search Agent
### 1. Connecting to a llama-stack server.
#### 1.1 Setting Up the Environment
- Import the necessary libraries.
- Defining user variables.
- Setting your ENV variables: Use the [`.env.example`](../../../.env.example) to create a new file called `.env` and ensure you add all the relevant environment variables
- Connection options
    - **Remote Server**: Set `remote=True` and provide the remote llama-stack endpoint.
    - **Local Server**: Set `remote=False` and provide the local llama-stack endpoint (default port: 8321).
    
    For detailed llama-stack server setup instructions, refer to:
    - [Remote Setup Guide](../../../kubernetes/README.md)
    - [Local Setup Guide](../../../local_setup_guide.md)

In [2]:
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client import LlamaStackClient
import os
import sys
sys.path.append('..')  
from src.utils import step_printer
from src.client_tools import get_location
from termcolor import cprint
from dotenv import load_dotenv
load_dotenv()

remote = True # Use the `remote` variable to switching between a local development environment and a remote kubernetes cluster.
stream_output = False # Set to True to stream the output of the agent.
inference_model = "ibm-granite/granite-3.2-8b-instruct"
print(f"Model: {inference_model}")

tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")  # necessary for websearch tool
if remote:
    base_url = os.getenv("REMOTE_BASE_URL")
else:
    base_url = "http://localhost:8321"

Model: ibm-granite/granite-3.2-8b-instruct


#### 1.2 Client Initialization
- Initialize the `LlamaStackClient` with the appropriate base URL and provider data.  
- Connect the llama-stack client to the llama-stack server using the base URL.

In [3]:
client = LlamaStackClient(
        base_url=base_url,
        provider_data={"tavily_search_api_key": tavily_search_api_key}
)

### 2. Configuring an agent for tool use.
- **Agent Initialization**: 

- Create an `Agent` instance with the desired model, instructions and tools."

- **Instructions**: The `instructions` parameter, also referred to as the system prompt, specifies the agent's role and behavior. In this example, the agent is configured as a helpful web search assistant. It is instructed to use a tool whenever a web search is required and to respond in a friendly and helpful tone.

- **Tools**: The `tools` parameter defines the tools available to the agent. In this case, the `builtin::websearch` tool is used, which enables the agent to perform web searches. This tool is essential for retrieving up-to-date information from the web. Internally, it leverages Tavily Search to execute the search queries efficiently.

- **How It Works**: When a user query is provided, the agent processes the input and determines whether a tool is required to fulfill the request. If the query involves retrieving information from the web, the agent invokes the `builtin::websearch` tool. The tool interacts with Tavily Search to fetch real-time data, which is then processed and returned to the user in a friendly and helpful tone. This workflow ensures that the agent can handle a wide range of queries effectively.

For more details on the `builtin::websearch` tool and its capabilities, refer to the [Llama-stack tools documentation](https://llama-stack.readthedocs.io/en/latest/building_applications/tools.html#web-search-providers). 

In [4]:
agent = Agent(
    client, 
    model=inference_model,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params={"max_tokens":4096}
)

### 3. Running the agent.
- Define user prompts to interact with the agent.
- Use the agent to create a session and process user queries.
- Display the agent's responses for each query.

In [5]:
user_prompts = [
    "What’s latest in OpenShift?",
    "Search for recent news articles about advancements in quantum computing.",
    "Find coffee shops near Boston Convention and Exhibition Center that are open now, have Wi-Fi.",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream_output
    )
    if stream_output:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: What’s latest in OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'latest updates on OpenShift'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'latest updates on OpenShift',
│   'top_k': [
│   │   {
│   │   │   'title': 'Red Hat unveils OpenShift 4.18: Enhanced security and virtualization ...',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-418',
│   │   │   'content': 'This article highlights the latest OpenShift 4.18 innovations and key enhancements. For a comprehensive list of updates and improvements, refer to the OpenShift 4.18 Release Notes. Enhance network flexibility with User Defined Networks and Border Gateway Protocol for pods and virtual machines',
│   │   │   'score': 0.85567063,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Experience the many enhancements in OpenShift 4.18',
│   │   │   'url': 'https://developers.redhat.com/blog/2025/02/25/whats-new-developers-red-hat-openshift-418',
│   │   │   'content': "For an in-depth review of all of the latest features, including updates for plug-ins and templates, refer to What's new in Red Hat Developer Hub 1.4. Red Hat OpenShift Virtualization Engine It is optimized for bare metal operating in a limited space and supports 128 cores per bare-metal socket pair.",
│   │   │   'score': 0.78963995,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift 4.17: What you need to know',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-417',
│   │   │   'content': 'Red Hat OpenShift 4.17 is now generally available. Based on Kubernetes 1.30 and CRI-O 1.30, OpenShift 4.17 features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in Red Hat OpenShift Platform Plus.These additions further accelerate innovation with OpenShift without compromising on',
│   │   │   'score': 0.767429,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift Container Platform Life Cycle Policy',
│   │   │   'url': 'https://access.redhat.com/support/policy/updates/openshift/',
│   │   │   'content': 'This life cycle page refers only to the latest major version of Red Hat OpenShift. ... and Important impact security updates and urgent-priority bug fixes for a predefined set of minor releases of Red Hat OpenShift. Extended Update Support - Long Life Add-on - Additional Term 1 enables customers to remain with the same minor release of Red Hat',
│   │   │   'score': 0.6466616,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "What's new in Red Hat OpenShift",
│   │   │   'url': 'https://www.redhat.com/en/whats-new-red-hat-openshift',
│   │   │   'content': 'Red Hat AI A portfolio for developing and deploying artificial intelligence solutions across the hybrid cloud.; Red Hat Enterprise Linux A flexible, stable operating system to support hybrid cloud innovation.; Red Hat OpenShift A container platform to build, modernize, and deploy applications at scale.; Red Hat Ansible Automation Platform A foundation for implementing enterprise-wide automation.',
│   │   │   'score': 0.31019315,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are the latest updates on OpenShift:

1. **Red Hat unveils OpenShift 4.18: Enhanced security and virtualization**
   - This version introduces improvements in network flexibility with User Defined Networks and Border Gateway Protocol for pods and virtual machines.
   - For a detailed list of updates and improvements, check out the [OpenShift 4.18 Release Notes](https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-418).

2. **Experience the many enhancements in OpenShift 4.18**
   - This article provides an overview of the latest features, including updates for plug-ins and templates.
   - For a comprehensive review, refer to [What's new in Red Hat Developer Hub 1.4](https://developers.redhat.com/blog/2025/02/25/whats-new-developers-red-hat-openshift-418).

3. **Red Hat OpenShift 4.17: What you need to know**
   - OpenShift 4.17, based on Kubernetes 1.30 and CRI-O 1.30, offers expanded control plane 

{
│   'query': 'recent news articles about advancements in quantum computing',
│   'top_k': [
│   │   {
│   │   │   'title': 'Quantum Computers News -- ScienceDaily',
│   │   │   'url': 'https://www.sciencedaily.com/news/computers_math/quantum_computers/',
│   │   │   'content': 'Quantum Computer Research. Read the latest news in developing quantum computers. ... Quantum Computers News. April 22, 2025 ... called a 3D photonic-crystal cavity that could enable transformative',
│   │   │   'score': 0.76427686,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Quantum Computing News -- ScienceDaily',
│   │   │   'url': 'https://www.sciencedaily.com/news/matter_energy/quantum_computing/',
│   │   │   'content': 'Quantum Computing News. Read the latest about the development of quantum computers. ... structure called a 3D photonic-crystal cavity that could enable transformative advancements in',
│   │   │   'score': 0.7458619,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Quantum computing | MIT News | Massachusetts Institute of Technology',
│   │   │   'url': 'https://news.mit.edu/topic/quantum-computing',
│   │   │   'content': 'A blueprint for making quantum computers easier to program. A CSAIL study highlights why it is so challenging to program a quantum computer to run a quantum algorithm, and offers a conceptual model for a more user-friendly quantum computer. April 16, 2024. Read full story →',
│   │   │   'score': 0.6811549,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Big Tech Has Entered Its Quantum Era With Major Advancements - Business ...',
│   │   │   'url': 'https://www.businessinsider.com/big-tech-officially-entered-quantum-era-major-advancements-2025-4?op=1',
│   │   │   'content': "Here's some of the biggest quantum news so far this year — and things to keep an eye on. ... that quantum computing will create between $450 billion and $850 billion of economic value globally",
│   │   │   'score': 0.62742686,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'New quantum phase observed for first time, with major implications for ...',
│   │   │   'url': 'https://www.livescience.com/technology/computing/scientists-observe-new-quantum-phase-that-could-have-major-implications-for-quantum-computing',
│   │   │   'content': 'The exotic quantum phase, predicted over half a century ago, could lead to advances in quantum computing, sensors and communication technology. Comments ( 0 ) ( )',
│   │   │   'score': 0.47504577,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some recent news articles about advancements in quantum computing:

1. **Quantum Computers News -- ScienceDaily**
   - [Read the latest news in developing quantum computers](https://www.sciencedaily.com/news/computers_math/quantum_computers/)
   - A team has developed a structure called a 3D photonic-crystal cavity that could enable transformative advancements in quantum computing.

2. **Quantum Computing News -- ScienceDaily**
   - [Read the latest about the development of quantum computers](https://www.sciencedaily.com/news/matter_energy/quantum_computing/)
   - The same team has again made headlines with their 3D photonic-crystal cavity, which could lead to significant advancements in quantum computing.

3. **Quantum computing | MIT News | Massachusetts Institute of Technology**
   - [A blueprint for making quantum computers easier to program](https://news.mit.edu/topic/quantum-computing)
   - A study from MIT

{
│   'query': 'coffee shops near Boston Convention and Exhibition Center open now with Wi-Fi',
│   'top_k': [
│   │   {
│   │   │   'title': 'Best Coffee near Boston Convention and Exhibition Center in ... - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?find_desc=Coffee&find_near=boston-convention-and-exhibition-center-boston',
│   │   │   'content': 'Reviews on Coffee near Boston Convention and Exhibition Center - Cocorico Boulangerie, Blue Bottle Coffee, Flour Bakery + Café, Tatte Bakery & Cafe, Caffè Nero, Tradesman Coffee Shop & Lounge, Seaport Bakery, Gracenote Coffee, Blank Street Coffee, Levain Bakery - Boston Seaport ... Open Now Offers Delivery Offers Takeout Free Wi-Fi Outdoor',
│   │   │   'score': 0.92343384,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'The Best 10 Cafes near MBTA - Hynes Convention Center in Boston, MA - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?cflt=cafes&find_near=mbta-hynes-convention-center-boston',
│   │   │   'content': 'Best Cafes near MBTA - Hynes Convention Center - Tatte Bakery & Cafe, Jonquils Cafe & Bakery, Beacon Hill Books & Cafe, Greystone Cafe, Bakery, and Provisions, Say Weekend, Cafe 26, Flour Bakery + Cafe, Madhouse Cafe, 180 Cafe ... Open Now Offers Delivery Offers Takeout Free Wi-Fi Outdoor Seating. 1. ... Black Owned Coffee Shop in Boston',
│   │   │   'score': 0.8488849,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Must-Visit Coffee Shops Near Our Convention Centers (Part II)',
│   │   │   'url': 'https://blog.signatureboston.com/must-visit-coffee-shops-near-our-convention-centers-part-ii',
│   │   │   'content': "Hynes. 176,480 square feet of adaptable exhibit space, auditorium with seating for 4,000+, 38 meeting rooms, and a 24,544-square-foot ballroom, all connected to free Wi-Fi and 3,100+ hotel rooms and 200 shops and restaurants in the heart of Boston's Back Bay.",
│   │   │   'score': 0.6922474,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Restaurants near Boston Convention and Exhibition Center',
│   │   │   'url': 'https://www.restaurantji.com/ma/places/boston-convention-and-exhibition-center/',
│   │   │   'content': 'Your guide to the best restaurants near Boston Convention and Exhibition Center. Explore local flavors and hidden gems with Restauranji. ... Coffee Shops • $ 7AM - 3PM 450 Summer Street, ... Not an open air rooftop restaurant but the view is nice if you are facing away from the convention center. A bit noisy, lots of noise coming from the',
│   │   │   'score': 0.5440511,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Best coffee shop near the convention center for a coffee ... - Reddit',
│   │   │   'url': 'https://www.reddit.com/r/boston/comments/1cljyz6/best_coffee_shop_near_the_convention_center_for_a/',
│   │   │   'content': "For a coffee enthusiast, it's definitely Gracenote, but you're gonna have to walk about 20 min from the convention center. Super nice folks run it and they always have the best indie music playing. If you wanna walk 30 min from the convention center, you could reach George Howell which is also fantastic.",
│   │   │   'score': 0.47475353,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some coffee shops near the Boston Convention and Exhibition Center that are currently open and offer Wi-Fi:

1. **Cocorico Boulangerie** - Reviews on Yelp mention that they are open now, offer delivery and takeout, and have free Wi-Fi. You can find them at [this link](https://www.yelp.com/search?find_desc=Coffee&find_near=boston-convention-and-exhibition-center-boston).

2. **Blue Bottle Coffee** - Also listed on Yelp, they are open now, offer delivery and takeout, and have free Wi-Fi. Check out their details [here](https://www.yelp.com/search?find_desc=Coffee&find_near=boston-convention-and-exhibition-center-boston).

3. **Flour Bakery + Café** - This place is also mentioned on Yelp. They are open now, offer delivery and takeout, and have free Wi-Fi. You can find more information [here](https://www.yelp.com/search?find_desc=Coffee&find_near=boston-convention-and-exhibition-center-boston).

4. **Tatte Bakery & Ca

## Output Analysis
Here, we can observe that the `builtin::websearch` tool is used to perform a web search. The outputs are displayed in the notebook with color-coded text to help interpret the process:

- **Blue Text**: Represents the user's input or query.
- **Yellow Text**: Displays the LLM's inference response. 
- **Green Text**: Indicates the tool execution process, such as the tool being called and the query being sent to the web search API.

Great! 
We can see that the model returned some relevant and recent information about OpenShift. This was only possible due to its ability to call tools like the web search, demonstrating the agent's capacity to retrieve real-time data effectively.

For the second query, the model fetched the latest news articles, including one from April 2025. This is particularly impressive since the Granite 3.2 8B model, released in February 2025, has a knowledge cutoff date of April 2024. This demonstrates the agent's capability to extend beyond its static knowledge base by dynamically retrieving current information.  

For the third query about nearby coffee shops, the agent effectively used the `builtin::websearch` tool to provide practical, location-specific results with valid coffee shops and links, highlighting its real-world applicability.  

## Prompt chaining with websearch tool and client tool

In this section, we demonstrate a more sophisticated use case that combines location detection with web search. Instead of explicitly providing location information, we'll:

1. **Automatic Location Detection**: Use the `get_location` client tool to automatically determine the user's current location
2. **Contextual Search**: Leverage the detected location to find relevant nearby places


For example, when a user asks "Find coffee shops near me that are open now, have Wi-Fi", the agent will:
- First detect the user's current location
- Then use that location to search for nearby coffee shops
- Present a comprehensive response with relevant options

This demonstrates how builtin websearch tool and client tool can work together to provide intelligent, context-aware responses without requiring explicit location input from the user.

In [7]:
agent = Agent(
    client, 
    model=inference_model,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    sampling_params={"max_tokens":4096}
)
user_prompts = [
    "Where am I?",
    "Find coffee shops near me that are open now, have Wi-Fi."
]
session_id = agent.create_session("web-session")  # for prompt chaining, queries must share the same session_id.
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream_output
    )

    if stream_output:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Dublin, Leinster, IE'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
You are currently in Dublin, Leinster, Ireland.

========== Query processing completed ========== 


Processing user query: Find coffee shops near me that are open now, have Wi-Fi.

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'coffee shops open now with Wi-Fi near Dublin'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'coffee shops open now with Wi-Fi near Dublin',
│   'top_k': [
│   │   {
│   │   │   'title': 'Best Coffee Shops Near Dublin, Dublin - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?find_desc=Best+Coffee+Shops&find_loc=Dublin',
│   │   │   'content': 'Reviews on Best Coffee Shops in Dublin, Republic of Ireland - Accents Coffee & Tea Lounge, The Bakehouse, Queen of Tarts, Beanhive, Kaph ... Best Coffee Shops Near Dublin, Dublin. Sort: Recommended. All. Price. Open Now Free Wi-Fi Outdoor Seating. 1.',
│   │   │   'score': 0.8909888,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'BEST of Dublin, Republic of Ireland 24/7 Coffee - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?find_desc=24/7+coffee&find_loc=Dublin',
│   │   │   'content': "Top 10 Best 24/7 Coffee in Dublin, Republic of Ireland - April 2025 - Yelp - Starbucks, Generator Dublin, Spar, Net House, Frank And Honest Gourmet Coffee Shop, Cloud Nine, Sophie's Bar, Tea Time Express, Supermacs ... Top 10 Best 24/7 coffee Near Dublin, Dublin. Sort: Recommended. All. Price. Open Now Offers Delivery Offers Takeout Free Wi-Fi",
│   │   │   'score': 0.86413884,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Best Free Wi-Fi Cafes in Dublin - UPDATED: March, 2025',
│   │   │   'url': 'https://laptopfriendlycafe.com/free-wifi-cafes-in-dublin',
│   │   │   'content': 'Check out the best cafes in Dublin where you can work or relax with free Wi-Fi! Find the perfect places to get stuff done or just chill out and enjoy yourself. ... culture, rich history, and friendly locals. For those craving a cozy spot to work or study while enjoying a cup of coffee, Dublin boasts a plethora of cafes with free wifi that cater',
│   │   │   'score': 0.71948266,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'The 10 Best Places To Grab A Coffee And Work In Dublin With Free Wi-Fi',
│   │   │   'url': 'https://lovindublin.com/dublin/the-10-best-places-to-grab-a-coffee-and-work-in-dublin-with-free-wi-fi',
│   │   │   'content': "In compiling this list, I've made sure each place met this criteria: a good strong free wi-fi connection, sockets to charge laptops or phone, and of course, good coffee. All of the venues selected below tick all the boxes and are spread out around the city, to give you an option for all situations.",
│   │   │   'score': 0.6852132,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '10 Best Places To Grab A Coffee And Work In Dublin With Free Wifi',
│   │   │   'url': 'https://lovindublin.com/dublin/best-places-work-remotely-dublin-free-wifi',
│   │   │   'content': "In terms of trendy spost that fit in with the tech culture of local internet giants like Google and Facebook this is the perfect spot. The industrial design and award winning coffee will keep you coming back here time and again. Coffee is very subjective but I'd say this is the best in Dublin. Full map (Google Maps link) Starbucks",
│   │   │   'score': 0.68064564,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some coffee shops near Dublin that are open now and offer Wi-Fi:

1. [Best Coffee Shops Near Dublin, Dublin - Yelp](https://www.yelp.com/search?find_desc=Best+Coffee+Shops&find_loc=Dublin) - This link provides reviews on some of the best coffee shops in Dublin, including Accents Coffee & Tea Lounge, The Bakehouse, Queen of Tarts, Beanhive, and Kaph.

2. [BEST of Dublin, Republic of Ireland 24/7 Coffee - Yelp](https://www.yelp.com/search?find_desc=24/7+coffee&find_loc=Dublin) - This link lists top 10 24/7 coffee spots in Dublin, including Starbucks, Generator Dublin, Spar, Net House, Frank And Honest Gourmet Coffee Shop, Cloud Nine, Sophie's Bar, Tea Time Express, and Supermacs.

3. [Best Free Wi-Fi Cafes in Dublin - UPDATED: March, 2025](https://laptopfriendlycafe.com/free-wifi-cafes-in-dublin) - This article provides a list of cafes in Dublin where you can work or relax with free Wi-Fi.

4. [The 10 Best Places T

### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

For example, when asked "Find coffee shops near me that are open now, have Wi-Fi", the agent:
1. **Reasons** it needs location information first
2. **Acts** by calling the `get_location` client tool
3. **Observes** the location result
4. **Reasons** about the next step
5. **Acts** by calling the `websearch` tool with observed location
6. **Observes** and processes the search results

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [8]:
agent = ReActAgent(
            client=client,
            model=inference_model,
            tools=[get_location, "builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params={"max_tokens":40960},
        )
user_prompts = [
    "Find coffee shops near me that are open now, have Wi-Fi."
]
session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream_output
    )
    if stream_output:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Find coffee shops near me that are open now, have Wi-Fi.

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
    "thought": "I need to find coffee shops near the user's current location that are open now and have Wi-Fi. I will use the `get_location` tool to get the user's current location and then use the `web_search` tool to find the coffee shops that meet the criteria.",
    "action": {
        "tool_name": "get_location",
        "tool_params": [{"name": "query", "value": "current location"}]
    },
    "answer": null
}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Dublin, Leinster, IE'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Now that I have the user's location, I will use the `web_search` tool to find coffee shops near Dublin, Leinster, IE that are open now and have Wi-Fi.",
    "action": {
        "tool_name": "web_search",
        "tool_params": [{"name": "query", "value": "coffee shops open now with Wi-Fi near Dublin, Leinster, IE"}]
    },
    "answer": null
}


---------- 📍 Step 4: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'coffee shops open now with Wi-Fi near Dublin, Leinster, IE',
│   'top_k': [
│   │   {
│   │   │   'title': 'Coffee Shops Dublin, Republic of Ireland - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?find_desc=coffee+shops&find_loc=Dublin',
│   │   │   'content': 'Top 10 Best Coffee Shops in Dublin, Republic of Ireland - January 2025 - Yelp - Shoe Lane Coffee, Beanhive, Queen of Tarts, Cocobrew, 9th Degree Coffee Roasters, Two Pups Coffee, The Bald Barista, The Bakehouse, Keoghs Café, Bread 41 ... Open Now Offers Delivery Offers Takeout Free Wi-Fi Outdoor Seating. 1. ... The Art Of Coffee, The Bald',
│   │   │   'score': 0.8315952,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'BEST of Dublin, Republic of Ireland Coffee - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?find_desc=coffee&find_loc=Dublin,+Ireland',
│   │   │   'content': 'Top 10 Best Coffee in Dublin, Republic of Ireland - April 2025 - Yelp - Shoe Lane Coffee, The Bald Barista, Queen of Tarts, Urbanity, Beanhive, Keoghs Café, Accents Coffee & Tea Lounge, The Bakehouse, Industry, Two Pups Coffee ... Top 10 Best coffee Near Dublin, Dublin ... Open Now Offers Delivery Offers Takeout Free Wi-Fi Outdoor Seating. 1.',
│   │   │   'score': 0.79883134,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'The 10 Best Cafés in County Dublin - Tripadvisor',
│   │   │   'url': 'https://www.tripadvisor.ie/Restaurants-g186604-c8-County_Dublin.html',
│   │   │   'content': "Best Coffee Shops in County Dublin, Province of Leinster. Cafés in County Dublin ... Open now. Open now. Features. Seating. Takeout. Wheelchair Accessible ... Restaurants near Windmill Lane Recording Studios Restaurants near The Irish Rock 'N' Roll Museum Experience Restaurants near Travelling Ireland Restaurants near 14 Henrietta Street",
│   │   │   'score': 0.6246834,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '10 Best Places To Grab A Coffee And Work In Dublin With Free Wifi',
│   │   │   'url': 'https://lovindublin.com/dublin/best-places-work-remotely-dublin-free-wifi',
│   │   │   'content': "In terms of trendy spost that fit in with the tech culture of local internet giants like Google and Facebook this is the perfect spot. The industrial design and award winning coffee will keep you coming back here time and again. Coffee is very subjective but I'd say this is the best in Dublin. Full map (Google Maps link) Starbucks",
│   │   │   'score': 0.5078369,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '10 Best Places To Grab A Coffee And Work In Dublin With Free Wi-Fi',
│   │   │   'url': 'https://lovin.ie/travel-1/cities/10-best-places-to-grab-a-coffee-and-work-in-dublin-with-free-wi-fi',
│   │   │   'content': "Open main menu. Search icon. Search icon. Open main menu All sections. Food and Drink; Counties; ... We've been looking for a good free dublin Wi-Fi list for some time and after not finding any we decided to research it a little more and create our own one. There are a few vitally important things when looking for a place with free Wi-Fi to",
│   │   │   'score': 0.40952834,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 5: InferenceStep ----------
🤖 Model Response:
{
    "thought": "I have found several coffee shops near Dublin, Leinster, IE that are open now and have Wi-Fi. I will now summarize the information and provide the user with the top results.",
    "action": null,
    "answer": "Here are some coffee shops near Dublin, Leinster, IE that are open now and have Wi-Fi: \n1. Shoe Lane Coffee \n2. Beanhive \n3. Queen of Tarts \n4. Cocobrew \n5. 9th Degree Coffee Roasters \n6. Two Pups Coffee \n7. The Bald Barista \n8. The Bakehouse \n9. Keoghs Café \n10. Bread 41 \n\nYou can find more details and reviews on Yelp. Enjoy your coffee!"
}

========== Query processing completed ========== 



## Key Takeaways
- This tutorial demonstrates how to set up and use websearch tool with AI Agent.
- It highlights the flexibility of switching between remote and local environments.
- By following these steps, users can quickly get started with AI Agents and explore its capabilities.
- It further showcased two advanced approach using prompting chaining and ReAct Agent.

For more advanced use cases, please refer to our other example Jupyter notebooks.